In [ ]:
#count genes with MITEs in full genome
import pandas as pd

df_blat = pd.read_csv('data/iwgsc_cn4/blat_95_95.csv', index_col=False, sep='\t')

df_genes = pd.read_csv('../../data/iwgsc_refseqv1.0_HighConf_2017Mar13.gff3', index_col=False, sep='\t')
df_genes.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df_genes = df_genes[df_genes.feature == 'gene'].reset_index(drop=True)

blat_genome = {}
for k,chromosome in df_genome.iterrows():
    blat_genome[chromosome.seqname] = df_blat[(df_blat.tName == chromosome.seqname)]
blat_genome['chrUn'] = df_blat[(df_blat.tName == 'chrUn')]
gene_with_mites_count = 0
gene_count = len(df_genes.index)
for k2,gene in df_genes.iterrows():
    curr_df = blat_genome[gene.seqname]
    res_1 = curr_df[(curr_df.tStart >= gene.start) & (curr_df.tStart <= gene.end)]
    res_2 = curr_df[(curr_df.tEnd >= gene.start) & (curr_df.tEnd <= gene.end)]
    if len(res_1.index) > 0 or len(res_2.index) > 0:
        gene_with_mites_count += 1
print(gene_with_mites_count)
print(gene_with_mites_count * 100 / gene_count)

In [ ]:
#count genes with MITEs per chromosome
import pandas as pd

df_blat = pd.read_csv('data/iwgsc_cn4/blat_95_95.csv', index_col=False, sep='\t')

df_genes = pd.read_csv('../../data/iwgsc_refseqv1.0_HighConf_2017Mar13.gff3', index_col=False, sep='\t')
df_genes.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df_genes = df_genes[df_genes.feature == 'gene'].reset_index(drop=True)

df_genome = pd.read_csv('data/iwgsc.gff3', index_col=False, sep='\t', header=None)
df_genome.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

blat_genome = {}
for k1,genome in df_genome.iterrows():
    curr_blat = df_blat[(df_blat.tName == genome.seqname)] 
    curr_genes = df_genes[(df_genes.seqname == genome.seqname)]
    gene_with_mites_count = 0
    gene_count = len(curr_genes.index)
    print(gene_count)
    for k2,gene in curr_genes.iterrows():
        res_1 = curr_blat[(curr_blat.tStart >= gene.start) & (curr_blat.tStart <= gene.end)]
        res_2 = curr_blat[(curr_blat.tEnd >= gene.start) & (curr_blat.tEnd <= gene.end)]
        if len(res_1.index) > 0 or len(res_2.index) > 0:
            gene_with_mites_count += 1
    print(genome.seqname,gene_with_mites_count)
    print(genome.seqname,gene_with_mites_count * 100 / gene_count)

In [ ]:
#count genes with MITEs per genome
import pandas as pd

df_blat = pd.read_csv('data/iwgsc_cn4/blat_95_95.csv', index_col=False, sep='\t')

df_genes = pd.read_csv('../../data/iwgsc_refseqv1.0_HighConf_2017Mar13.gff3', index_col=False, sep='\t')
df_genes.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df_genes = df_genes[df_genes.feature == 'gene'].reset_index(drop=True)

df_genome = pd.read_csv('data/iwgsc.gff3', index_col=False, sep='\t', header=None)
df_genome.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
print('starting...')
blat_genome = {}
for k,chromosome in df_genome.iterrows():
    blat_genome[chromosome.seqname] = df_blat[(df_blat.tName == chromosome.seqname)]
print('proccessing genomes...')
for genome in ['A','B','D']:
    curr_genes = df_genes[(df_genes.seqname.str.contains(genome))]
    gene_with_mites_count = 0
    gene_count = len(curr_genes.index)
    print(gene_count)
    for k2,gene in curr_genes.iterrows():
        curr_df = blat_genome[gene.seqname]
        res_1 = curr_df[(curr_df.tStart >= gene.start) & (curr_df.tStart <= gene.end)]
        res_2 = curr_df[(curr_df.tEnd >= gene.start) &  (curr_df.tEnd <= gene.end)]
        if len(res_1.index) > 0 or len(res_2.index) > 0:
            if k2 % 1000 == 0:
                print(k2,gene_with_mites_count)
            gene_with_mites_count += 1
    print(genome,gene_with_mites_count)
    print(genome,gene_with_mites_count * 100 / gene_count)

In [ ]:
#percentage of genome covered by MITEs
import pandas as pd

def merge_overlap(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []
    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            # test for intersection between lower and higher:
            # we know via sorting that lower[0] <= higher[0]
            if higher[0] <= lower[1]:
                upper_bound = max(lower[1], higher[1])
                merged[-1] = (lower[0], upper_bound)  # replace by merged interval
            else:
                merged.append(higher)
    return merged

df_blat = pd.read_csv('data/iwgsc_cn4/blat_95_95.csv', index_col=False, sep='\t')

df_genome = pd.read_csv('data/iwgsc.gff3', index_col=False, sep='\t', header=None)
df_genome.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
intervals = {}
for k,v in df_blat.iterrows():
    genome = v.tName
    if not genome in intervals:
        intervals[genome] = []
    intervals[genome].append((v.tStart,v.tEnd))
    
print("a")
percs_genome = {}
total_sum = 0
for k,v in df_genome.iterrows():
    genome = v.seqname[4:6]
    if not genome in percs_genome:
        percs_genome[genome] = 0
    current_interval = merge_overlap(intervals[v.seqname])
    interval_sum = 0
    for i in current_interval:
        interval_sum += max(i) - min(i)
    perc = interval_sum * 100 / v.end
    print(v.seqname, perc)
    #calculate avg for full genomes
    percs_genome[genome] += perc
    total_sum += perc
    
for genome in ['A','B','D']:
    print(genome, percs_genome[genome] / 7)
print('Total', total_sum / 21)

In [ ]:
#count genes per genome
import pandas as pd
df_genes = pd.read_csv('../../data/iwgsc_refseqv1.0_HighConf_2017Mar13.gff3', index_col=False, sep='\t')
df_genes.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df_genes = df_genes[df_genes.feature == 'gene']
total_genes = len(df_genes.index)
print('total genes', total_genes)
s = df_genes.seqname.value_counts()
genome = {}
for k in s.index.tolist():
    gen = k[4:6]
    if not gen in genome:
        genome[gen] = s[k]
    else:
        genome[gen] += s[k]
print(genome)

In [ ]:
#calculate MITE and gene density
import pandas as pd
import sys
df_blast = pd.read_csv('data/iwgsc_cn4/blat_95_95.csv', index_col=False, sep='\t')
df_genome = pd.read_csv('data/iwgsc.gff3', index_col=False, sep='\t', header=None)
df_genome.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
total_bp = 0
for k,v in df_genome.iterrows():
    total_bp += v.end

total_mites = len(df_blast.index)
total_mb = total_bp / 1000 / 1000
total_mb_total = total_mb
total_gb = total_mb / 1000 
print ('Total:','&',int(total_mb),'&',total_mites,'&',int(total_mites / total_mb))

genome_sizes = {}
for k,v in df_genome.iterrows():
    df_blast_seq = df_blast[df_blast.tName == v.seqname]
    total_mites = len(df_blast_seq.index)
    total_mb = v.end / 1000 / 1000
    seq = v.seqname.replace('chr','')
    if not seq[1:2] in genome_sizes:
        genome_sizes[seq[1:2]] = total_mb
    else:
        genome_sizes[seq[1:2]] += total_mb
    print (seq,'&',int(total_mb),'&',total_mites,'&',int(total_mites / total_mb))

for genome in ['A','B','D']:
    df_blast_seq = df_blast[df_blast.tName.str.contains(genome)]
    total_mites = len(df_blast_seq.index)
    print (genome,'&', int(genome_sizes[genome]),'&',total_mites,'&',int(total_mites / genome_sizes[genome]))
print('*' * 10)
    
    
df_genes = pd.read_csv('../../data/iwgsc_refseqv1.0_HighConf_2017Mar13.gff3', index_col=False, sep='\t')
df_genes.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
df_genes = df_genes[df_genes.feature == 'gene']
total_genes = len(df_genes.index)
print ('Total Genes:',total_genes)
print ('Total Genes per MB:',total_genes / total_mb_total)